# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):

# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
host = '127.0.0.1'

from cassandra.cluster import Cluster
try: 
    cluster = Cluster([host]) #If you have a locally installed Apache Cassandra instance
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)


#### Create Keyspace

In [6]:
# Create a keyspace if it does not already exist
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS btproject2 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace("btproject2")
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
# Creating table for Query 1
create_table_qry1 = "create table if not exists song_play_library "
create_table_qry1 = create_table_qry1 + "(session_id int, item_in_session int, artist_name text, song_title text, \
    song_length float, primary key (session_id, item_in_session))"
try:
    session.execute(create_table_qry1)
except Exception as e:
    print(e)
         

In [10]:
# Confirm that the table is created by querying metadata
desc_qry = ("select * from system_schema.columns where keyspace_name = 'btproject2' and table_name = 'song_play_library'")
try:
    rows = session.execute(desc_qry)
    pt = PrettyTable(['Keyspace', 'Table', 'Column', 'Clustering Order', 'Kind', 'Position', 'Type'])
    for row in rows:
        pt.add_row([row.keyspace_name, row.table_name, row.column_name, row.clustering_order, row.kind, row.position, row.type])

    print(pt)
except Exception as e:
    print(e)


+------------+-------------------+-----------------+------------------+---------------+----------+-------+
|  Keyspace  |       Table       |      Column     | Clustering Order |      Kind     | Position |  Type |
+------------+-------------------+-----------------+------------------+---------------+----------+-------+
| btproject2 | song_play_library |   artist_name   |       none       |    regular    |    -1    |  text |
| btproject2 | song_play_library | item_in_session |       asc        |   clustering  |    0     |  int  |
| btproject2 | song_play_library |    session_id   |       none       | partition_key |    0     |  int  |
| btproject2 | song_play_library |   song_length   |       none       |    regular    |    -1    | float |
| btproject2 | song_play_library |    song_title   |       none       |    regular    |    -1    |  text |
+------------+-------------------+-----------------+------------------+---------------+----------+-------+


In [11]:
# Insert data into 'song_play_library' table from the combined CSV file picking the necessary columns
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # print(type(line[8]), type(line[3]), type(line[0]), type(line[9]), type(line[5]))
        insert_table_qry1 = "insert into song_play_library (session_id, item_in_session, artist_name, song_title, song_length) "
        insert_table_qry1 = insert_table_qry1 + "values (%s, %s, %s, %s, %s)"
        session.execute(insert_table_qry1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
# Query 1:  Select the artist, song title and song's length in the music app history 
# that was heard during sessionId = 338, and itemInSession = 4
select_table_qry1 = "select session_id, item_in_session, artist_name, song_title, song_length \
                    from song_play_library \
                    WHERE session_id = 338 \
                    and item_in_session = 4"
try:
    rows = session.execute(select_table_qry1)
except Exception as e:
    print(e)

pt = PrettyTable(['Session ID', 'Item in Session', 'Artist', 'Song', 'Length'])
for row in rows:
    pt.add_row([row.session_id, row.item_in_session, row.artist_name, row.song_title, row.song_length])

print(pt)

+------------+-----------------+-----------+---------------------------------+--------------------+
| Session ID | Item in Session |   Artist  |               Song              |       Length       |
+------------+-----------------+-----------+---------------------------------+--------------------+
|    338     |        4        | Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+------------+-----------------+-----------+---------------------------------+--------------------+


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
# Creating table for Query 2
create_table_qry2 = "create table if not exists song_playlist_session "
create_table_qry2 = create_table_qry2 + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, \
    user_fname text, user_lname text, primary key ((user_id, session_id), item_in_session))"
try:
    session.execute(create_table_qry2)
except Exception as e:
    print(e)


In [14]:
# Confirm that the table is created by querying metadata
desc_qry = ("select * from system_schema.columns where keyspace_name = 'btproject2' and table_name = 'song_playlist_session'")
try:
    rows = session.execute(desc_qry)
    pt = PrettyTable(['Keyspace', 'Table', 'Column', 'Clustering Order', 'Kind', 'Position', 'Type'])
    for row in rows:
        pt.add_row([row.keyspace_name, row.table_name, row.column_name, row.clustering_order, row.kind, row.position, row.type])

    print(pt)
except Exception as e:
    print(e)


+------------+-----------------------+-----------------+------------------+---------------+----------+------+
|  Keyspace  |         Table         |      Column     | Clustering Order |      Kind     | Position | Type |
+------------+-----------------------+-----------------+------------------+---------------+----------+------+
| btproject2 | song_playlist_session |   artist_name   |       none       |    regular    |    -1    | text |
| btproject2 | song_playlist_session | item_in_session |       asc        |   clustering  |    0     | int  |
| btproject2 | song_playlist_session |    session_id   |       none       | partition_key |    1     | int  |
| btproject2 | song_playlist_session |    song_title   |       none       |    regular    |    -1    | text |
| btproject2 | song_playlist_session |    user_fname   |       none       |    regular    |    -1    | text |
| btproject2 | song_playlist_session |     user_id     |       none       | partition_key |    0     | int  |
| btprojec

In [15]:
# Insert data into 'song_playlist_session' table from the combined CSV file picking the necessary columns
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        # print(type(line[10]), type(line[8]), type(line[3]), type(line[0]), type(line[9]), type(line[1]), type(line[4]))
        insert_table_qry2 = "insert into song_playlist_session (user_id, session_id, item_in_session, artist_name, song_title, user_fname, user_lname) "
        insert_table_qry2 = insert_table_qry2 + "values (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(insert_table_qry2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [16]:
# Query 2: Select only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
# for userid = 10, sessionid = 182

select_table_qry2 = "select artist_name,song_title, user_fname, user_lname from song_playlist_session WHERE user_id = 10 and session_id = 182"
try:
    rows = session.execute(select_table_qry2)
except Exception as e:
    print(e)

pt = PrettyTable(['Artist', 'Song', 'User First Name', 'User Last Name'])
for row in rows:
    pt.add_row([row.artist_name, row.song_title, row.user_fname, row.user_lname])

print(pt)


+-------------------+------------------------------------------------------+-----------------+----------------+
|       Artist      |                         Song                         | User First Name | User Last Name |
+-------------------+------------------------------------------------------+-----------------+----------------+
|  Down To The Bone |                  Keep On Keepin' On                  |      Sylvie     |      Cruz      |
|    Three Drives   |                     Greece 2000                      |      Sylvie     |      Cruz      |
| Sebastien Tellier |                      Kilometer                       |      Sylvie     |      Cruz      |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |      Sylvie     |      Cruz      |
+-------------------+------------------------------------------------------+-----------------+----------------+


In [17]:
# Create table for Query 3
create_table_qry3 = "create table if not exists user_play_library "
create_table_qry3 = create_table_qry3 + "(song_title text, user_id int, user_fname text, user_lname text, primary key (song_title, user_id))"
try:
    session.execute(create_table_qry3)
except Exception as e:
    print(e)
                    

In [18]:
# Confirm that the table is created by querying metadata
desc_qry = ("select * from system_schema.columns where keyspace_name = 'btproject2' and table_name = 'user_play_library'")
try:
    rows = session.execute(desc_qry)
    pt = PrettyTable(['Keyspace', 'Table', 'Column', 'Clustering Order', 'Kind', 'Position', 'Type'])
    for row in rows:
        pt.add_row([row.keyspace_name, row.table_name, row.column_name, row.clustering_order, row.kind, row.position, row.type])

    print(pt)
except Exception as e:
    print(e)


+------------+-------------------+------------+------------------+---------------+----------+------+
|  Keyspace  |       Table       |   Column   | Clustering Order |      Kind     | Position | Type |
+------------+-------------------+------------+------------------+---------------+----------+------+
| btproject2 | user_play_library | song_title |       none       | partition_key |    0     | text |
| btproject2 | user_play_library | user_fname |       none       |    regular    |    -1    | text |
| btproject2 | user_play_library |  user_id   |       asc        |   clustering  |    0     | int  |
| btproject2 | user_play_library | user_lname |       none       |    regular    |    -1    | text |
+------------+-------------------+------------+------------------+---------------+----------+------+


In [19]:
# Insert data into 'user_play_library' table from the combined CSV file picking the necessary columns
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        # print(type(line[10]), type(line[8]), type(line[3]), type(line[0]), type(line[9]), type(line[1]), type(line[4]))
        insert_table_qry3 = "insert into user_play_library (song_title, user_id, user_fname, user_lname) "
        insert_table_qry3 = insert_table_qry3 + "values (%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(insert_table_qry3, (line[9], int(line[10]), line[1], line[4]))

In [20]:
# Query 3: Select every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'
select_table_qry3 = "select user_fname, user_lname from user_play_library WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(select_table_qry3)
except Exception as e:
    print(e)

pt = PrettyTable(['User First Name', 'User Last Name'])
for row in rows:
    pt.add_row([row.user_fname, row.user_lname])

print(pt)


+-----------------+----------------+
| User First Name | User Last Name |
+-----------------+----------------+
|    Jacqueline   |     Lynch      |
|      Tegan      |     Levine     |
|       Sara      |    Johnson     |
+-----------------+----------------+


### Drop the tables before closing out the sessions

In [21]:
## TO-DO: Drop the table before closing out the sessions
table_list = ['song_play_library', 'user_play_library', 'song_playlist_session']
for table_name in table_list:
    drop_table_qry = "drop table if exists " + table_name
    try:
        rows = session.execute(drop_table_qry)
    except Exception as e:
        print(e)


### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()